# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [10]:
!pip install langchain
!pip install langchain-openai


   ---------------------------------------- 0.0/884.5 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/884.5 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 884.5/884.5 kB 3.3 MB/s eta 0:00:00


In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [12]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [13]:
prompt = ChatPromptTemplate.from_template( 
    "Can you provide a detailed and engaging description for a new product called '{product_name}'? "
    "The description should highlight its features, such as fitness tracking, heart rate monitoring, GPS, and long battery life. "
    "The tone should be friendly and appealing to tech-savvy consumers."
 
)

In [14]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\jaime\AppData\Local\Temp\ipykernel_10136\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [15]:
product = "SuperSmart Watch"
chain.run(product)

C:\Users\jaime\AppData\Local\Temp\ipykernel_10136\1918983842.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"Introducing the SuperSmart Watch - the ultimate companion for the modern, tech-savvy individual who is always on the go! Packed with cutting-edge features, this sleek and stylish smart watch is designed to help you stay connected, motivated, and on top of your health and fitness goals.\n\nStay on track with your workouts with the built-in fitness tracking feature that monitors your steps, distance, and calories burned throughout the day. The heart rate monitoring functionality provides real-time data on your heart rate, helping you optimize your workouts and keep tabs on your overall health.\n\nNever get lost again with the built-in GPS that allows you to track your routes and navigate with ease. Whether you're exploring a new city or hitting the trails for a run, the SuperSmart Watch has got you covered.\n\nAnd the best part? With a long-lasting battery life, you can wear your SuperSmart Watch all day and night without having to worry about constantly charging it. This means you can 

## SimpleSequentialChain

In [16]:
from langchain.chains import SimpleSequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Can you provide a detailed and engaging description for a new product called '{product_name}'? "
    "The description should highlight its features, such as fitness tracking, heart rate monitoring, GPS, and long battery life. "
    "The tone should be friendly and appealing to tech-savvy consumers."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [19]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the following product description: '{product_description}', can you create an engaging marketing copy that emphasizes its unique selling points and appeals to potential buyers?"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [20]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [21]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Introducing the SuperSmart Watch, the ultimate companion for the modern tech-savvy individual who values both style and functionality. 

Stay ahead of the game with our cutting-edge fitness tracking feature, which allows you to monitor your steps, calories burned, and distance traveled. Take your workouts to the next level with heart rate monitoring, ensuring you stay in the optimal heart rate zone for maximum results. 

Never get lost again with built-in GPS, so you can easily track your routes and explore new trails without any worries. And rest assured with our long-lasting battery life, so you can keep moving all day without constantly recharging.

But that's not all! The SuperSmart Watch also offers customizable watch faces, notifications for calls and messages, and water resistance for added versatility. Whether you're hitting the gym, going for a run, or just navigating your daily life, the SuperSmart Watch has got you covered.

Up

"Are you ready to take control of your health and fitness goals? Look no further than the SuperSmart Watch - the ultimate blend of style and functionality for the modern individual on the go.\n\nKeep track of your steps, calories burned, and distance traveled with our cutting-edge fitness tracking feature. With heart rate monitoring, you can push yourself to the max and stay in the zone for optimal results. And with built-in GPS, you'll never have to worry about getting lost on your adventures.\n\nBut that's not all - customize your watch face, receive notifications for calls and messages, and enjoy water resistance for added versatility. With long-lasting battery life, you can stay connected and active all day long without missing a beat.\n\nDon't settle for ordinary - upgrade to the SuperSmart Watch and experience the power of smart technology at your fingertips. Take your smartwatch experience to the next level and transform the way you live your life. Get yours today and make every

**Repeat the above twice for different products**

## SequentialChain

In [22]:
from langchain.chains import SequentialChain

In [36]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review into Italian: {review}."
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated_review" #Give a name to your output
                    )

In [37]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review: {translated_review}."
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="review_summary" #give a name to this output
                    )


In [39]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
   "Translate the following summary into Spanish: {review_summary}."
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_summary"
                      )

In [40]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Based on the translated review and summary, write a follow-up message for the reviewer. "
    "Translated review: {translated_review}. "
    "Summary: {review_summary}."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [41]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translated_review", "review_summary", "translated_summary", "followup_message"],
    verbose=True
)

In [42]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'translated_review': 'Trovo il gusto mediocre. La schiuma non tiene, è strano. Compro gli stessi prodotti in commercio e il gusto è molto migliore... Vecchio stock o contraffazione!?',
 'review_summary': 'Il recensore ritiene che il prodotto abbia un gusto mediocre e che la schiuma non sia di buona qualità. Sospetta che potrebbe trattarsi di un vecchio stock o di una contraffazione, poiché ha acquistato gli stessi prodotti in commercio con un gusto molto migliore.',
 'translated_summary': 'El crítico considera que el producto tiene un sabor mediocre y que la espuma no es de buena calidad. Sospecha que podría ser un viejo stock o una falsificación, ya que ha comprado los mismos productos en el mercado con un sabor mucho mejor.',
 'followup_message': 'Dear reviewer,\n\nThank you for taking the time to leave feed

**Repeat the above twice for different products or reviews**

## Router Chain

In [43]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [44]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [45]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [46]:
llm = ChatOpenAI(temperature=0)

In [47]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [48]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [49]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [50]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [51]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\jaime\AppData\Local\Temp\ipykernel_10136\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [52]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [53]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [54]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [55]:
# Prueba con varias preguntas
result1 = chain.run("What is black body radiation?")
print("Physics Chain Output:", result1)

result2 = chain.run("What is 2 + 2?")
print("Math Chain Output:", result2)

result3 = chain.run("Why does every cell in our body contain DNA?")
print("Biology/Default Chain Output:", result3)



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
Physics Chain Output: Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power.


> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.
Math Chain Output: The answer to 2 + 2 is 4.


> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'